In [2]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np

# Store excel file locations to variables (change it as per your path to file)
file_2012=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\5. Section 2 Part 2c Own livestock work during last 12m\2012_s2p2c_f.xlsx"
file_2013=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\5. Section 2 Part 2c Own livestock work during last 12m\2013_s2p2c_f.csv"
file_2014=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\5. Section 2 Part 2c Own livestock work during last 12m\2014_s2p2c_f.csv"

# Read excel files
df_2012 = pd.read_excel(file_2012)
df_2013 = pd.read_csv(file_2013)
df_2014 = pd.read_csv(file_2014)


In [3]:
'''
This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.
For example in the roaster data for 2013 rq21 and rq23 are not the same as rq21 and rq23 in 2014 data, but they have the same variable names. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.

'''

# Rename columns in df_2012
df_2012.rename(columns={
    'S2P2CQ0': 's2p2c_qa',
    'pid': 'r_pid',
    'S2P2CQ1A': 's2p2c_q1',
    'S2P2CQ1B': 's2p2c_q2',
    'S2P2CQ3A': 's2p2c_q6a',
    'S2P2CQ3B': 's2p2c_q6b',
    'S2P2CQ2A': 'O_LvsProd_HpD',
    'S2P2CQ2B': 'O_LvsProd_Td',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'UC_ID': 'UC_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)

# Rename columns in df_2013
df_2013.rename(columns={
    's2p2c_q1a': 's2p2c_q1',
    's2p2c_q1b': 's2p2c_q2',
    's2p2c_q2a': 's2p2c_q3',
    's2p2c_q2b': 's2p2c_q4',
    's2p2c_q3a': 's2p2c_q5',
    's2p2c_q3b': 's2p2c_q6',
    's2p2c_q4a': 's2p2c_q7',
    's2p2c_q4b': 's2p2c_q8',
    's2p2c_q5a': 's2p2c_q9',
    's2p2c_q5b': 's2p2c_q10',
    's2p2c_q6a': 's2p2c_q11',
    's2p2c_q6b': 's2p2c_q12'
}, inplace=True)

# df_2014 doesn't need renaming as it is the reference

In [4]:
# Updated mappings

mapping_2012 = [
    'hid', 'round', 's2p2c_qa', 'r_pid', 's2p2c_q1', 's2p2c_q2',
    None, None, None, None, None, None, None,
    's2p2c_q6a', 's2p2c_q6b', 'O_LvsProd_HpD', 'O_LvsProd_Td',
    'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID'
]

mapping_2013 = [
    'hid', 'round', None, 'r_pid', 's2p2c_q1', 's2p2c_q2', 's2p2c_q3', 's2p2c_q4', 
    's2p2c_q5', 's2p2c_q6', 's2p2c_q7', 's2p2c_q8', 's2p2c_q9', 's2p2c_q10', 's2p2c_q11', 
    's2p2c_q12', None, None, None, None, None, None, None
]

mapping_2014 = [
    'hid', 'round', 's2p2c_qa', 'r_pid', 's2p2c_q1', 's2p2c_q2', 's2p2c_q3', 's2p2c_q4', 
    's2p2c_q5', 's2p2c_q6', 's2p2c_q7', 's2p2c_q8', 's2p2c_q9', 's2p2c_q10', 's2p2c_q11', 
    's2p2c_q12', None, None, None, None, None, None, None
]


In [5]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [6]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [7]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['hid', 'round', 'r_pid', 's2p2c_qa', 's2p2c_q1', 's2p2c_q2', 'O_LvsProd_HpD', 'O_LvsProd_Td', 's2p2c_q6a', 's2p2c_q6b', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column hid
Appending data for column round
Appending data for column s2p2c_qa
Appending data for column r_pid
Appending data for column s2p2c_q1
Appending data for column s2p2c_q2
Appending data for column s2p2c_q6a
Appending data for column s2p2c_q6b
Appending data for column O_LvsProd_HpD
Appending data for column O_LvsProd_Td
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending data for column UC_ID
Appending data for column M_ID
Processing DataFrame with columns: ['Unnamed: 0', 'hid', 'round', 'r_pid', 's2p2c_q1', 's2p2c_q2', 's2p2c_q3', 's2p2c_q4', 's2p2c_q5', 's2p2c_q6', 's2p2c_q7', 's2p2c_q8', 's2p2c_q9', 's2p2c_q10', 's2p2c_q11', 's2p2c_q12']
Appending data for column hid
Appending data for column round
Appending dat

In [8]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'hid': 'HID',
    'round': 'Survey_Round',
    's2p2c_qa': 'O_LvsW',
    'r_pid': 'PID',
    's2p2c_q1': 'O_LvsC_HpD',
    's2p2c_q2': 'O_LvsC_Td',
    's2p2c_q3': 'O_Milk_HpD',
    's2p2c_q4': 'O_Milk_Td',
    's2p2c_q5': 'O_DC_HpD',
    's2p2c_q6': 'O_DC_Td',
    's2p2c_q7': 'O_Graz_HpD',
    's2p2c_q8': 'O_Graz_Td',
    's2p2c_q9': 'O_MedC_HpD',
    's2p2c_q10': 'O_MedC_Td',
    's2p2c_q11': 'O_OLvsA_HpD',
    's2p2c_q12': 'O_OLvsA_Td'
}

merged_df.rename(columns=rename_mapping, inplace=True)
'''
# Drop redundant columns
merged_df.drop(merged_df.columns[merged_df.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
merged_df.drop(merged_df.columns[merged_df.columns.str.contains(' ', case=False)], axis=1, inplace=True)
'''
# Save the merged dataframe to a CSV file
merged_df.to_csv('3. merged_EnI_Own Livestock Work During Past 12m.csv', index=False)
